In [1]:
import yfinance as yF
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

In [2]:
#------

In [3]:
#Start and End date need to be at least one month greater than a year apart. 
start ="2022-01-01"
end = "2023-06-01"

In [4]:
#List of tickers used for the strategy.
ticker_s = ['XLI', 'XLB', 'XLP', 'XLK', 'XLY', 'XLV', 'XLF', 'XLE', 'XLU', 'XLC', 'XLRE', 'PDBC', 'GLD', 'TLT', 'IEF', 'TIP', 'LTPZ']

In [5]:
#------

In [8]:
def SMA_Above(ticker_symbol):
    for column in ticker_symbol:
        ticker_symbol[column+'_SMA'] = ticker_symbol[column].rolling(168).mean()
        if ticker_symbol.iloc[-1][column+'_SMA'] > ticker_symbol.iloc[-1][column]:
            ticker_symbol.drop(column, axis=1, inplace=True)
    
    regex_str = '_SMA'
    ticker_symbol = ticker_symbol.drop(ticker_symbol.columns[ticker_symbol.columns.str.contains(regex_str)], axis=1)
    global column_names
    column_names = ticker_symbol.columns.values.tolist()

In [9]:
def get_momentum_multiple(current):
    global momentum_matrix
    #Major issues with the monthly outcomes not being accurate. 
    #------
    #1 Month
    month_1 = current.tail(1).copy()
    #3 Month
    month_3 = current.tail(3).copy()
    #6 Month
    month_6 = current.tail(6).copy()
    #9 Month
    month_9 = current.tail(9).copy()
    #12 Month
    month_12 = current.tail(12).copy()
    #Month 1 Perf.
    month_1_per = month_1
    month_1 = (month_1_per*100).round(2)
    #Month 3,6,9,12 Perf.
    df_list = month_3, month_6, month_9, month_12
    for dataframe in df_list:
        for column in dataframe:
            dataframe[column] = ((1 + dataframe[column]).cumprod()-1)
            dataframe[column] = (dataframe[column]*100).round(2)    
    #------
    #Average
    momentum_matrix = pd.concat([month_1.tail(1), month_3.tail(1), month_6.tail(1), month_9.tail(1), month_12.tail(1)], axis=0)
    momentum_matrix = momentum_matrix.mean(axis=0).round(2)
    #Removes Assets with 0 or negative momentum
    momentum_matrix = momentum_matrix.drop(momentum_matrix[momentum_matrix <= 0].index)
    #Only selects the top 5 assets
    momentum_matrix = momentum_matrix.nlargest(n=5, keep='first')

In [10]:
#------

In [11]:
#SMA Data
#Removes assets that are below thier 168 day or 8 month SMA. 
df = yF.download(ticker_s, start=start, end=end)['Adj Close']
SMA_Above(df)

[*********************100%***********************]  17 of 17 completed


In [12]:
column_names

['GLD', 'IEF', 'LTPZ', 'TIP', 'XLB', 'XLC', 'XLI', 'XLK', 'XLP', 'XLV', 'XLY']

In [13]:
#------

In [14]:
#Momentum
#Selects the top 5 assets based on momentum and removes assets with momentum <= 0. 
ticker_s = column_names
df1 = yF.download(ticker_s, start=start, end=end)['Adj Close']
df1 = (df1.pct_change() +1)[1:].resample('M').prod()
df1 = df1 - 1
get_momentum_multiple(df1)

[*********************100%***********************]  11 of 11 completed


In [15]:
momentum_matrix

XLK    13.41
XLC    10.67
GLD     8.61
XLI     2.66
XLP     2.49
dtype: float64

In [16]:
path = 'C:/Users/monic/Documents/Portfolio Data/Strategy Asset Data'
output_file = os.path.join(path,'US_Sector_Assets.csv')
momentum_matrix.to_csv(output_file, index=True)